# AML_STR_Detector_Lite_V1 🇩🇪  
*Rule-Based Suspicious Transaction Detection Engine with FATF, Basel & BaFin Alignment*


# Step 2: Import Dependencies

In [4]:
import json
from datetime import datetime

# Step 3: Customer Dataset

In [6]:
customers = [
    {
        "customer_id": "CUST001",
        "name": "Rahul Kumar",
        "financial_inclusion_flag": True,
        "risk_band": "Low",
        "onboarded_via": "PM Jan Dhan Yojana",
        "transactions": [
            {"txn_id": "TXN101", "rule_triggered": "RDT", "base_weight": 30, "channel": "IMPS"},
            {"txn_id": "TXN102", "rule_triggered": "MCPT", "base_weight": 40, "channel": "IMPS"}
        ]
    },
    {
        "customer_id": "CUST002",
        "name": "Naveen Rao",
        "financial_inclusion_flag": False,
        "risk_band": "Medium",
        "onboarded_via": "Standard Onboarding",
        "transactions": [
            {"txn_id": "TXN201", "rule_triggered": "SMF", "base_weight": 75, "channel": "SWIFT"}
        ]
    },
    {
        "customer_id": "CUST003",
        "name": "Fatima Zehra",
        "financial_inclusion_flag": False,
        "risk_band": "High",
        "onboarded_via": "Offshore Entity",
        "transactions": [
            {"txn_id": "TXN301", "rule_triggered": "MCPT", "base_weight": 45, "channel": "NEFT"},
            {"txn_id": "TXN302", "rule_triggered": "SMF", "base_weight": 70, "channel": "NEFT"}
        ]
    },
    {
        "customer_id": "CUST004",
        "name": "Michael Hans",
        "financial_inclusion_flag": False,
        "risk_band": "Medium",
        "onboarded_via": "Digital KYC",
        "transactions": [
            {"txn_id": "TXN401", "rule_triggered": "RDT", "base_weight": 30, "channel": "RTGS"}
        ]
    }
]

# Step 4: Rule Weights, Severities, Red Flags, BaFin Tags

In [8]:
rule_weights = {
    "RDT": 30,
    "MCPT": 40,
    "SMF": 75
}

rule_severity = {
    "RDT": "Low",
    "MCPT": "Medium",
    "SMF": "High"
}

red_flag_lookup = {
    "RDT": "Recurring round-figure transactions",
    "MCPT": "Mismatch between transaction pattern and customer profile",
    "SMF": "Structuring detected via fragmented transactions"
}

bafin_risk_lookup = {
    "RDT": "Value-based anomaly",
    "MCPT": "Risk-profile deviation",
    "SMF": "Fragmentation of funds"
}

# Step 5: Risk Score Adjustment (based on Financial Inclusion flag)

In [10]:
def get_adjusted_weight(rule, fi_flag, base_weight):
    if fi_flag and rule in ["RDT", "MCPT"]:
        return base_weight * 0.5  # More lenient on Financial Inclusion customers
    return base_weight

# Step 6: Alert Classification

In [12]:
def classify_alert(score, fi_flag):
    if fi_flag and score < 50:
        return "Soft Alert - Proportionate Review"
    elif score >= 80:
        return "Hard Alert - Escalation"
    else:
        return "Moderate Alert - Queue for L1 Analyst"

# Step 7: STR Narrative Generator (Transaction Articulation)

In [14]:
def generate_articulation(name, triggered_rules, channel, risk_band):
    rules_str = ", ".join(triggered_rules)
    return (f"Customer '{name}' triggered rules: {rules_str} via {channel}. "
            f"Behavior inconsistent with risk profile '{risk_band}'.")

# Step 8: STR Alert Builder Function

In [16]:
def build_str_alert(customer):
    triggered_rules = [txn["rule_triggered"] for txn in customer["transactions"]]
    channel = customer["transactions"][0]["channel"]
    
    total_score = sum(
        get_adjusted_weight(txn["rule_triggered"], customer["financial_inclusion_flag"], txn["base_weight"])
        for txn in customer["transactions"]
    )
    
    return {
        "customer_profile": {
            "customer_id": customer["customer_id"],
            "name": customer["name"],
            "financial_inclusion_flag": customer["financial_inclusion_flag"],
            "risk_band": customer["risk_band"],
            "onboarded_via": customer["onboarded_via"]
        },
        "trigger_rules": triggered_rules,
        "rule_severity": max(
            [rule_severity[r] for r in triggered_rules],
            key=lambda sev: ["Low", "Medium", "High"].index(sev)
        ),
        "risk_score": round(total_score, 2),
        "alert_classification": classify_alert(total_score, customer["financial_inclusion_flag"]),
        "red_flags": [red_flag_lookup[r] for r in triggered_rules],
        "bafin_risk_category": [bafin_risk_lookup[r] for r in triggered_rules],
        "transaction_articulation": generate_articulation(
            customer["name"], triggered_rules, channel, customer["risk_band"]
        ),
        "compliance_references": {
            "FATF": ["Rec 10", "Rec 20"],
            "Basel": "Principle 3 - Transaction Monitoring",
            "BaFin": "Section 6 GwG – Risk Management"
        },
        "str_status": (
            "Filed" if total_score >= 80 else
            "Queued for Filing" if total_score >= 50 else
            "Deferred"
        ),
        "timestamp": datetime.now().isoformat(),
        "channel": channel
    }

# Step 9: Generate STR Alerts for All Customers

In [18]:
alerts = []

for customer in customers:
    alert = build_str_alert(customer)
    alerts.append(alert)

# Pretty print the results
import pprint
pprint.pprint(alerts)

[{'alert_classification': 'Soft Alert - Proportionate Review',
  'bafin_risk_category': ['Value-based anomaly', 'Risk-profile deviation'],
  'channel': 'IMPS',
  'compliance_references': {'BaFin': 'Section 6 GwG – Risk Management',
                            'Basel': 'Principle 3 - Transaction Monitoring',
                            'FATF': ['Rec 10', 'Rec 20']},
  'customer_profile': {'customer_id': 'CUST001',
                       'financial_inclusion_flag': True,
                       'name': 'Rahul Kumar',
                       'onboarded_via': 'PM Jan Dhan Yojana',
                       'risk_band': 'Low'},
  'red_flags': ['Recurring round-figure transactions',
                'Mismatch between transaction pattern and customer profile'],
  'risk_score': 35.0,
  'rule_severity': 'Medium',
  'str_status': 'Deferred',
  'timestamp': '2025-07-01T16:38:00.158767',
  'transaction_articulation': "Customer 'Rahul Kumar' triggered rules: RDT, "
                              'MCPT via

# Step 10: Export Alerts to JSON

In [20]:
with open("alerts_output.json", "w") as f:
    json.dump(alerts, f, indent=2)

print("STR alerts exported to alerts_output.json")

STR alerts exported to alerts_output.json


# Step 10: Export Alerts to Excel

In [35]:
import pandas as pd

# Flatten the alert dictionary into tabular format for Excel
flat_alerts = []

for alert in alerts:
    flat_alerts.append({
        "Customer ID": alert["customer_profile"]["customer_id"],
        "Customer Name": alert["customer_profile"]["name"],
        "Risk Band": alert["customer_profile"]["risk_band"],
        "Financial Inclusion": alert["customer_profile"]["financial_inclusion_flag"],
        "Onboarded Via": alert["customer_profile"]["onboarded_via"],
        "Rules Triggered": ", ".join(alert["trigger_rules"]),
        "Rule Severity": alert["rule_severity"],
        "Risk Score": alert["risk_score"],
        "Alert Classification": alert["alert_classification"],
        "Red Flags": ", ".join(alert["red_flags"]),
        "BaFin Risk Category": ", ".join(alert["bafin_risk_category"]),
        "Narrative": alert["transaction_articulation"],
        "Compliance Reference - FATF": ", ".join(alert["compliance_references"]["FATF"]),
        "Compliance Reference - Basel": alert["compliance_references"]["Basel"],
        "Compliance Reference - BaFin": alert["compliance_references"]["BaFin"],
        "STR Status": alert["str_status"],
        "Channel": alert["channel"],
        "Timestamp": alert["timestamp"]
    })

# Create DataFrame
df_alerts = pd.DataFrame(flat_alerts)

# Export to Excel
df_alerts.to_excel("alerts_output.xlsx", index=False)

print("STR alerts exported successfully to 'alerts_output.xlsx'")

STR alerts exported successfully to 'alerts_output.xlsx'


## AML_STR_Detector_Lite_V1 Summary

This notebook presents a compact, rule-based Suspicious Transaction Reporting (STR) engine with regulatory alignment to FATF, Basel Principles, and BaFin (Germany's financial regulator). It includes:

### Features:
- Rule-based AML detection using RDT (Round Dollar Transactions), MCPT (Mismatched Customer Profile & Transaction Type), and SMF (Structuring)
- Risk score adjustment based on financial inclusion flags to ensure proportionate surveillance
- Alert classification logic that categorizes alerts into:
  - Soft Alert – Proportionate Review
  - Moderate Alert – Queue for L1 Analyst
  - Hard Alert – Escalation

### Output Fields:
- Customer Profile
- Triggered Rules and Corresponding Severities
- Adjusted Risk Score
- Alert Classification Outcome
- Identified Red Flags (Behavioral Indicators)
- BaFin Risk Categories
- Compliance References:
  - FATF Recommendations (e.g., Rec 10, Rec 20)
  - Basel Principle 3 – Transaction Monitoring
  - BaFin: Section 6 GwG – Risk Management
- Narrative Summary of Transaction Behavior (Transaction Articulation)
- STR Filing Status (Filed, Queued, or Deferred)
- Timestamp of Alert Generation
- Channel through which the transaction was conducted

This notebook is designed for rapid demonstration, extension, and deployment across financial institutions working toward enhance AML compliance frameworks.
nnel of Execution
annel of Execution
